In [1]:
import pandas as pd
import re

# Tentativa de Automação

In [2]:
dias = ["SEG", "TER", "QUA", "QUI", "SEX", "SAB"]

intervalos_inicio = {"07:30": 1, "08:20" : 2, "09:20": 3, "10:10": 4, "11:00": 5, "11:50": 6,
                    "13:30": 7, "14:20" : 8, "15:20": 9, "16:10": 10, "17:00": 11, "17:50": 12,
                    "18:50": 13, "19:40": 14, "20:40": 15, "21:30": 16}

intervalos_fim = {"08:19": 1, "09:09" : 2, "09:10" : 2, "10:09": 3, "10:59": 4, "11:49": 5, "11:50": 5, "12:40": 6, "12:39": 6,
                 "14:19": 7, "15:09" : 8, "15:10" : 8, "16:09": 9, "16:59": 10, "17:49": 11, "17:50": 11 ,"18:40": 12, "18:39": 12,
                  "19:39": 13, "20:29": 14, "20:30": 14, "21:29": 15, "22:20": 16}

In [3]:
def separate_time_slots(inp_str):
    # Regular expression to find day and time intervals
    pattern = re.compile(r'\b(?:[A-Z]{3} \d{2}:\d{2}~\d{2}:\d{2})\b')

    # Find all occurrences of day and time intervals using the pattern
    result = re.findall(pattern, inp_str)

    return result

In [4]:
def timeslot_to_index(timeslot_str):
    # Regular expression to extract the time interval from the time slot
    pattern = re.compile(r'([A-Z]{3}) (\d{2}:\d{2})~(\d{2}:\d{2})')
    
    # Find the time interval using the pattern
    match = re.search(pattern, timeslot_str)

    day = match.group(1)
    first_slot = match.group(2)
    last_slot = match.group(3)
    first_index = intervalos_inicio.get(first_slot)
    last_index = intervalos_fim.get(last_slot)
    return day, first_index, last_index

In [5]:
def process_timeslots(inp):
    lecture_ts = separate_time_slots(inp)
    day_indexes = {}
    for ts in lecture_ts:
        day, first_index, last_index = timeslot_to_index(ts)
        if last_index == None:
            print(ts)
        indexes = list(range(first_index, last_index + 1))
        if day in day_indexes:
            day_indexes[day].extend(indexes)
        else:
            day_indexes[day] = indexes
    return day_indexes

In [6]:
# Apply the process_timeslots function to the "Horarios" column and create/update corresponding columns
def sheet_analisys(df):
    for index, row in df.iterrows():
        day_indexes = process_timeslots(row["HORáRIOS"])
        for day, indexes in day_indexes.items():
            df.at[index, day] = ", ".join(str(i) for i in indexes)
    return df

## Teste

In [7]:
import os

dir_path = "../../data/og-sol/"
dir = os.listdir(dir_path)

for file_name in dir:
    if file_name.endswith(".xls") or file_name.endswith(".xlsx"):
        file_path = os.path.join(dir_path, file_name)
        print(file_path)
        df = pd.read_excel(file_path)
        df2 = sheet_analisys(df.copy())
        new_order = ["CóDIGO", "TURMA","CóD. DISCIPLINA","COMPONENTE CURRICULAR","HORáRIOS","PROFESSORES DO DIáRIO","SEG","TER","QUA","QUI","SEX","SAB"] 
        df2 = df2.reindex(columns=new_order)
        df2.to_excel(dir_path+"corrigido-vao-se-foderem-corpo-administrativo-do-if/"+file_name,index=False)
        display(df2)

../../data/og-sol/TCC-Instancia-20181.xlsx


,CóDIGO,TURMA,CóD. DISCIPLINA,COMPONENTE CURRICULAR,HORáRIOS,PROFESSORES DO DIáRIO,SEG,TER,QUA,QUI,SEX,SAB
0,114936,02CDC20I.3IESP,RV.BCC011,PROGRAMAçãO ORIENTADA A OBJETOS,[SEX 10:10~11:50 SEX 13:30~15:09],EDGAR PEREIRA DOS SANTOS,NaN,NaN,NaN,NaN,"4, 5, 7, 8",NaN
1,104699,20181.02ADM112.1V,RV.ADM01,REDAçãO DE TEXTOS OFICIAIS,[QUA 13:30~15:09 QUA 15:20~16:09],WILCIENE NUNES DO VALE,NaN,NaN,"7, 8, 9",NaN,NaN,NaN
2,104696,20181.02ADM112.1V,RV.ADM02,ÉTICA,[SEG 15:20~17:50],MILTON BERNARDES FERREIRA,"9, 10, 11",NaN,NaN,NaN,NaN,NaN
3,104694,20181.02ADM112.1V,RV.ADM03,CENáRIOS ECONôMICOS E MERCADOS,[SEX 15:20~17:50],HAIHANI SILVA PASSOS,NaN,NaN,NaN,NaN,"9, 10, 11",NaN
4,104700,20181.02ADM112.1V,RV.ADM04,SOCIOLOGIA APLICADA à ADMINISTRAçãO,[QUA 16:10~17:50],ROBERTO EDUARDO CASTILLO PIZARRO,NaN,NaN,"10, 11",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
588,99546,20181.02ZOO20I.9V,RV.HUM209,GESTãO DE NEGóCIOS,[QUA 13:30~15:09 QUA 15:20~16:09],CARLOS ANTONIO CARDOSO SOBRINHO,NaN,NaN,"7, 8, 9",NaN,NaN,NaN
589,99542,20181.02ZOO20I.9V,RV.ZOO216,AVICULTURA,[SEG 13:30~15:09 SEG 15:20~16:59],FABIANA RAMOS DOS SANTOS,"7, 8, 9, 10",NaN,NaN,NaN,NaN,NaN
590,99544,20181.02ZOO20I.9V,RV.ZOO218,CAPRINOCULTURA E OVINOCULTURA,[TER 13:30~15:09 TER 15:20~16:59],ELIS APARECIDO BENTO,NaN,"7, 8, 9, 10",NaN,NaN,NaN,NaN
591,99547,20181.02ZOO20I.9V,RV.ZOO219,JULGAMENTO E PREPARO DE ANIMAIS PARA EXPOSIçãO,[QUI 13:30~15:09],ADRIANO CARVALHO COSTA,NaN,NaN,NaN,"7, 8",NaN,NaN


../../data/og-sol/TCC-Instancia-20182.xlsx


,CóDIGO,TURMA,CóD. DISCIPLINA,COMPONENTE CURRICULAR,HORáRIOS,PROFESSORES DO DIáRIO,SEG,TER,QUA,QUI,SEX,SAB
0,111622,20182.02ADM112.1V,RV.ADM01,REDAçãO DE TEXTOS OFICIAIS,[TER 16:10~17:50],WILCIENE NUNES DO VALE,NaN,"10, 11",NaN,NaN,NaN,NaN
1,111619,20182.02ADM112.1V,RV.ADM02,ÉTICA,[SEX 13:30~15:09 SEX 15:20~16:09],THOMAZ JOEZER HERLER,NaN,NaN,NaN,NaN,"7, 8, 9",NaN
2,111617,20182.02ADM112.1V,RV.ADM03,CENáRIOS ECONôMICOS E MERCADOS,[TER 13:30~15:09 TER 15:20~16:09],HAIHANI SILVA PASSOS,NaN,"7, 8, 9",NaN,NaN,NaN,NaN
3,111623,20182.02ADM112.1V,RV.ADM04,SOCIOLOGIA APLICADA à ADMINISTRAçãO,[QUA 15:20~16:59],ROBERTO EDUARDO CASTILLO PIZARRO,NaN,NaN,"9, 10",NaN,NaN,NaN
4,111620,20182.02ADM112.1V,RV.ADM401,GESTãO EMPRESARIAL,[TER 15:20~16:59 QUA 13:30~15:09],LUCIENE DE OLIVEIRA GUERRA,NaN,"9, 10","7, 8",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
554,110914,20182.02ZOO20I.8M,RV.HUM205,SOCIOLOGIA E EXTENSãO RURAL,[QUI 09:20~11:50],JESIEL SOUZA SILVA,NaN,NaN,NaN,"3, 4, 5",NaN,NaN
555,110911,20182.02ZOO20I.8M,RV.HUM207,ECONOMIA RURAL,[SEX 07:30~09:09 SEX 09:20~10:09],SAMANTHA REZENDE MENDES,NaN,NaN,NaN,NaN,"1, 2, 3",NaN
556,110910,20182.02ZOO20I.8M,RV.ZOO217,BOVINOCULTURA DE CORTE E BUBALINOCULTURA,[TER 07:30~09:09 QUI 07:30~09:09],KAREN MARTINS LEãO,NaN,"1, 2",NaN,"1, 2",NaN,NaN
557,110912,20182.02ZOO20I.8M,RV.ZOO225,NUTRIçãO DE RUMINANTES,[QUA 07:30~09:09 QUA 09:20~10:09],KáTIA CYLENE GUIMARãES,NaN,NaN,"1, 2, 3",NaN,NaN,NaN


../../data/og-sol/TCC-Instancia-20191.xlsx


,CóDIGO,TURMA,CóD. DISCIPLINA,COMPONENTE CURRICULAR,HORáRIOS,PROFESSORES DO DIáRIO,SEG,TER,QUA,QUI,SEX,SAB
0,130164,20191.02ADI11N.1N,RV.ADM01,REDAçãO DE TEXTOS OFICIAIS,[SEG 20:40~22:20 QUI 19:40~20:29],MARISTELA SILVESTRE SILVA,"15, 16",NaN,NaN,14,NaN,NaN
1,130161,20191.02ADI11N.1N,RV.ADM02,ÉTICA,[QUA 18:50~20:29 QUA 20:40~21:29],VIVIANE DE FáTIMA CARDOSO OLIVEIRA,NaN,NaN,"13, 14, 15",NaN,NaN,NaN
2,130159,20191.02ADI11N.1N,RV.ADM03,CENáRIOS ECONôMICOS E MERCADOS,[SEX 20:40~22:20],ADEMIR XAVIER DE SOUZA,NaN,NaN,NaN,NaN,"15, 16",NaN
3,130165,20191.02ADI11N.1N,RV.ADM04,SOCIOLOGIA APLICADA à ADMINISTRAçãO,[TER 20:40~22:20],VIVIANE DE FáTIMA CARDOSO OLIVEIRA,NaN,"15, 16",NaN,NaN,NaN,NaN
4,130162,20191.02ADI11N.1N,RV.ADM401,GESTãO EMPRESARIAL,[TER 18:50~20:29 QUA 21:30~22:20],VIVIANE DE FáTIMA CARDOSO OLIVEIRA,NaN,"13, 14",16,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
616,123441,20191.02ZOO20I.9,RV.HUM209,GESTãO DE NEGóCIOS,[QUA 09:20~11:50],CARLOS ANTONIO CARDOSO SOBRINHO,NaN,NaN,"3, 4, 5",NaN,NaN,NaN
617,123437,20191.02ZOO20I.9,RV.ZOO216,AVICULTURA,[SEX 07:30~09:09 SEX 09:20~10:59],FABIANA RAMOS DOS SANTOS,NaN,NaN,NaN,NaN,"1, 2, 3, 4",NaN
618,123439,20191.02ZOO20I.9,RV.ZOO218,CAPRINOCULTURA E OVINOCULTURA,[SEG 08:20~09:09 SEG 09:20~11:50],ELIS APARECIDO BENTO,"2, 3, 4, 5",NaN,NaN,NaN,NaN,NaN
619,123442,20191.02ZOO20I.9,RV.ZOO219,JULGAMENTO E PREPARO DE ANIMAIS PARA EXPOSIçãO,[QUA 07:30~09:09],ANA PAULA CARDOSO GOMIDE; BRUNO FRAUZINO RIBEI...,NaN,NaN,"1, 2",NaN,NaN,NaN


../../data/og-sol/TCC-Instancia-20192.xlsx


,CóDIGO,TURMA,CóD. DISCIPLINA,COMPONENTE CURRICULAR,HORáRIOS,PROFESSORES DO DIáRIO,SEG,TER,QUA,QUI,SEX,SAB
0,137689,20192.02ADI11N.1N,RV.ADM01,REDAçãO DE TEXTOS OFICIAIS,[SEG 21:30~22:20 QUI 20:40~22:20],FRANCIELLE FERREIRA DE FREITAS,16,NaN,NaN,"15, 16",NaN,NaN
1,137686,20192.02ADI11N.1N,RV.ADM02,ÉTICA,[TER 18:50~20:29 QUA 18:50~19:39],FRANCIELLE FERREIRA DE FREITAS,NaN,"13, 14",13,NaN,NaN,NaN
2,137684,20192.02ADI11N.1N,RV.ADM03,CENáRIOS ECONôMICOS E MERCADOS,[QUA 19:40~20:29 SEX 21:30~22:20],ADEMIR XAVIER DE SOUZA,NaN,NaN,14,NaN,16,NaN
3,137690,20192.02ADI11N.1N,RV.ADM04,SOCIOLOGIA APLICADA à ADMINISTRAçãO,[SEX 19:40~20:29 SEX 20:40~21:29],FRANCIELLE FERREIRA DE FREITAS,NaN,NaN,NaN,NaN,"14, 15",NaN
4,137687,20192.02ADI11N.1N,RV.ADM401,GESTãO EMPRESARIAL,[SEG 18:50~20:29 SEG 20:40~21:29],FREDERICO DE MORAIS BORGES,"13, 14, 15",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
569,133506,20192.02ZOO20I.ESPECIAL,RV.BIO201,BIOLOGIA CELULAR,[QUA 13:30~16:09 QUI 13:30~16:09],MARIA ANDREIA CORREA MENDONçA; VALDNéA CASAGRA...,NaN,NaN,"7, 8, 9","7, 8, 9",NaN,NaN
570,133509,20192.02ZOO20I.ESPECIAL,RV.EXA201,GEOMETRIA ANALíTICA E ÁLGEBRA LINEAR,[QUI 07:30~10:59],ALINE GOBBI DUTRA; TIAGO PEREIRA GUIMARãES,NaN,NaN,NaN,"1, 2, 3, 4",NaN,NaN
571,133511,20192.02ZOO20I.ESPECIAL,RV.QUI205,QUíMICA GERAL E ANALíTICA,[QUA 09:20~11:49 QUI 16:10~17:49],ROBERTA DE ALMEIDA CARVALHO,NaN,NaN,"3, 4, 5","10, 11",NaN,NaN
572,135687,20192.02ZOO20I.ESPECIAL,RV.ZOO202,ANATOMIA ANIMAL,[QUA 13:30~16:59],BRUNO FRAUZINO RIBEIRO CAMILO,NaN,NaN,"7, 8, 9, 10",NaN,NaN,NaN


../../data/og-sol/TCC-Instancia-20221.xlsx


,CóDIGO,TURMA,CóD. DISCIPLINA,COMPONENTE CURRICULAR,HORáRIOS,PROFESSORES DO DIáRIO,SEG,TER,QUA,QUI,SEX,SAB
0,183444,20221.02ADM11N.1N,RV.ADM01,REDAçãO DE TEXTOS OFICIAIS,[SEG 20:40~22:20],SANDRA MARA SANTOS LEMOS,"15, 16",NaN,NaN,NaN,NaN,NaN
1,183441,20221.02ADM11N.1N,RV.ADM02,ÉTICA,[SEX 18:50~20:29],LORENA AZEVEDO LIMA,NaN,NaN,NaN,NaN,"13, 14",NaN
2,183439,20221.02ADM11N.1N,RV.ADM03,CENáRIOS ECONôMICOS E MERCADOS,[QUA 20:40~22:20],SAMANTHA REZENDE MENDES,NaN,NaN,"15, 16",NaN,NaN,NaN
3,183445,20221.02ADM11N.1N,RV.ADM04,SOCIOLOGIA APLICADA à ADMINISTRAçãO,[SEG 18:50~20:29],EMIVAL DA CUNHA RIBEIRO,"13, 14",NaN,NaN,NaN,NaN,NaN
4,183442,20221.02ADM11N.1N,RV.ADM401,GESTãO EMPRESARIAL,[TER 18:50~20:29 TER 20:40~22:20],KENNEDY DE ARAúJO BARBOSA,NaN,"13, 14, 15, 16",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
447,182527,20221.02ZOO20I.7MV,GRAD.TPOA.01,TECNOLOGIA DE PRODUTOS DE ORIGEM ANIMAL,[QUA 07:30~09:09 QUA 09:20~10:09],MARCO ANTôNIO PEREIRA DA SILVA,NaN,NaN,"1, 2, 3",NaN,NaN,NaN
448,182533,20221.02ZOO20I.9MV,GRAD.AVICULT.01,AVICULTURA,[SEG 13:30~15:09 SEG 15:20~16:09],FABIANA RAMOS DOS SANTOS,"7, 8, 9",NaN,NaN,NaN,NaN,NaN
449,182534,20221.02ZOO20I.9MV,GRAD.BL.01,BOVINOCULTURA DE LEITE,[QUA 13:30~15:09 QUA 15:20~16:09],TIAGO PEREIRA GUIMARãES,NaN,NaN,"7, 8, 9",NaN,NaN,NaN
450,182543,20221.02ZOO20I.9MV,GRAD.EMPREEDAGRO.01,EMPREENDEDORISMO NO AGRONEGóCIO,[QUA 16:10~18:39],LORENA AZEVEDO LIMA,NaN,NaN,"10, 11, 12",NaN,NaN,NaN


# Analise de dados

In [ ]:
def sexto_horario(matrix):
    sexto = 0

    for row in matrix:
        if row[5] == 1:
            sexto = sexto + 1
        if row[11] == 1:
            sexto = sexto + 1
    return sexto

In [ ]:
def sabado(matrix):
    if 

In [ ]:
def janela(matrix):
    return

In [ ]:
dir_path = "../../data/og-sol/corrigido-vao-se-foderem-corpo-administrativo-do-if/"

dir = os.listdir(dir_path)

for file_name in dir:
    if file_name.endswith(".xls") or file_name.endswith(".xlsx"):
        file_path = os.path.join(dir_path, file_name)
        print(file_path)
        df = pd.read_excel(file_path)
        
        # Create a list of unique teachers
        teachers = df["PROFESSORES DO DIáRIO"].unique()
        
        # Iterate through the DataFrame and mark the slots in the matrix
        for _, row in df.iterrows():
            teacher = row["PROFESSORES DO DIáRIO"]
            days = ["SEG", "TER", "QUA", "QUI", "SEX", "SAB"]
            matrix = [[0 for _ in range(16)] for _ in range(len(days))]
            for day in days:
                if day in row and pd.notnull(row[day]):
                    slots = row[day].split(",")
                    for slot in slots:
                        matrix[days.index(day)][int(int(slot)-1)] = 1
            print(teacher)
            display(matrix)

            


../../data/og-sol/corrigido-vao-se-foderem-corpo-administrativo-do-if/TCC-Instancia-20181.xlsx
EDGAR PEREIRA DOS SANTOS


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

WILCIENE NUNES DO VALE


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

MILTON BERNARDES FERREIRA


...

RAQUEL MARIA PRADO


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

RúBIA CRISTINA ARANTES MARQUES


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

RAQUEL MARIA PRADO
